# Circular Random Projection
- This is a software experiment to check out how to do circular random projection
- The idea is that instead of randomizing a huge matrix, we can make a smaller seed matrix and populate the larger matrix with smaller circular permutations of the larger matrix

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Useful Functions

In [2]:
# Random indexing function to make an NxM matrix
def random_indexing(N, M):
    # Create aranged indices
    matrix = np.arange(N * M)
    # Shuffle the matrix
    np.random.shuffle(matrix)
    matrix = matrix.reshape(N, M)
    # Create the bipolar matrix
    matrix[matrix < (N * M) // 2] = -1
    matrix[matrix >= (N * M) // 2] = 1
    return matrix

In [3]:
def numbered_indexing(N, M):
    # Create an NxM matrix with numbers from 0 to N*M-1
    matrix = np.arange(N * M).reshape(N, M)
    return matrix

In [4]:
def circular_block(seed_matrix):
    return np.roll(seed_matrix.flatten(), shift=1).reshape(seed_matrix.shape)

In [5]:
def matrix_expansion(sub_matrix, X, Y):
    # Copy sub matrix
    base_matrix = sub_matrix

    # Stitching the bigger matrix
    blocks = []

    # Horizontal expansion
    for i in range(Y):
        row_blocks = []
        # Vertical expansion
        for j in range(X):
            base_matrix = circular_block(base_matrix)
            row_blocks.append(base_matrix)
        
        # Stack vertically
        blocks.append(np.vstack(row_blocks))

    # Stack horizontally
    final_matrix = np.hstack(blocks)

    return final_matrix

# Creating a Simple Test Case
- Let's try for now a simple input vector of $1 \times 16$ multiplied to a matrix $16 \times 16$ elements.
- However the matrix $16 \times 16$ is created using sub $4 \times 4$ matrices that were pre-generated with 1s and -1s
- There is a random indexing function to ensure the number of 1s and -1s are equal

In [6]:
# Create base vector
vector = numbered_indexing(1, 16)
print(f"Base vector: {vector}")

# Create sub-matrix
sub_matrix = numbered_indexing(4, 4)
print(f"Sub-matrix:\n{sub_matrix}")

Base vector: [[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]]
Sub-matrix:
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


- The example below shows how the indices change per sub-matrix that is permuted

In [7]:
print(f"Sub-matrix iter 0:\n{sub_matrix}")

permute_sub_matrix = circular_block(sub_matrix)
print(f"Sub-matrix iter 1:\n{permute_sub_matrix}")

permute_sub_matrix = circular_block(permute_sub_matrix)
print(f"Sub-matrix iter 2:\n{permute_sub_matrix}")

permute_sub_matrix = circular_block(permute_sub_matrix)
print(f"Sub-matrix iter 3:\n{permute_sub_matrix}")

Sub-matrix iter 0:
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
Sub-matrix iter 1:
[[15  0  1  2]
 [ 3  4  5  6]
 [ 7  8  9 10]
 [11 12 13 14]]
Sub-matrix iter 2:
[[14 15  0  1]
 [ 2  3  4  5]
 [ 6  7  8  9]
 [10 11 12 13]]
Sub-matrix iter 3:
[[13 14 15  0]
 [ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


- The matrix below shows the indices of how those permutations are stitched together.
- So the sub-matrices from the larger block needs to be permutated vertically first before moving horizontally.
- The pattern of the indices should be sufficient to show how the permutations work.

In [8]:
expanded_matrix = matrix_expansion(sub_matrix, 4, 4)
expanded_matrix

array([[15,  0,  1,  2, 11, 12, 13, 14,  7,  8,  9, 10,  3,  4,  5,  6],
       [ 3,  4,  5,  6, 15,  0,  1,  2, 11, 12, 13, 14,  7,  8,  9, 10],
       [ 7,  8,  9, 10,  3,  4,  5,  6, 15,  0,  1,  2, 11, 12, 13, 14],
       [11, 12, 13, 14,  7,  8,  9, 10,  3,  4,  5,  6, 15,  0,  1,  2],
       [14, 15,  0,  1, 10, 11, 12, 13,  6,  7,  8,  9,  2,  3,  4,  5],
       [ 2,  3,  4,  5, 14, 15,  0,  1, 10, 11, 12, 13,  6,  7,  8,  9],
       [ 6,  7,  8,  9,  2,  3,  4,  5, 14, 15,  0,  1, 10, 11, 12, 13],
       [10, 11, 12, 13,  6,  7,  8,  9,  2,  3,  4,  5, 14, 15,  0,  1],
       [13, 14, 15,  0,  9, 10, 11, 12,  5,  6,  7,  8,  1,  2,  3,  4],
       [ 1,  2,  3,  4, 13, 14, 15,  0,  9, 10, 11, 12,  5,  6,  7,  8],
       [ 5,  6,  7,  8,  1,  2,  3,  4, 13, 14, 15,  0,  9, 10, 11, 12],
       [ 9, 10, 11, 12,  5,  6,  7,  8,  1,  2,  3,  4, 13, 14, 15,  0],
       [12, 13, 14, 15,  8,  9, 10, 11,  4,  5,  6,  7,  0,  1,  2,  3],
       [ 0,  1,  2,  3, 12, 13, 14, 15,  8,  9, 10,

# Ideal Matrix Multiplication
- In the ideal matrix multiplication, it's simply the vector multiplied by the matrix

In [9]:
ideal_matmul = np.matmul(vector, expanded_matrix)
ideal_matmul

array([[804, 924, 980, 972, 772, 876, 916, 892, 804, 892, 916, 876, 900,
        972, 980, 924]])

# Tiled Circular Matrix Projection
- So the idea of the tiled circular matrix projection is that we don't need to store the entire permuted matrices but start only from a seed matrix, multiply things one by one.
- In this case suppose we have an input vector of size $1 \times 16$
- Then we have a sub-matrices of size $4 \times 4$ and the thing is we want to slide and circular permute this unto the input vector as if we did the multiplication of the vector and the expanded $16 \times 16$ matrix.
- Since we have $4 \times 4$ sub-matrices, then it's sufficient to say to cut the vector into four $1 \times 4$ sub-vectors.
- Then we multiply the sub-vectors to the column-wise sub-matrices first as it produces $1 \times 4$ sub-vectors for the output and we eventually concatenate them to make the desired $1 \times 16$ output vector.

## First, cut input vector into sub-vectors
- Technically easier done to just reshape the vector into $4 \times 4$ where each row is one sub-vector.

In [10]:
print(f"Orig vector:\n{vector}")

split_vector = vector.reshape(4, 4)
print(f"Split vector:\n{split_vector}")

Orig vector:
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]]
Split vector:
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


# Iterative Circular Projections
- Since we only, supposedly, store the seed sub-matrix, then we need to iterate through the permutations one by one.
- In this example we need to iterate 16 times.

In [11]:
# Initialize expected output
expected_output = np.zeros((4, 4), dtype=int)

# Iterate through the permuted sub-matrices
permuted_sub_matrix = sub_matrix.copy()

for i in range(4):
    for j in range(4):
        # Get the permuted sub-matrix
        permuted_sub_matrix = circular_block(permuted_sub_matrix)
        
        # Perform matrix multiplication only on the sub-matrix and sub-vector
        result = np.matmul(split_vector[j, :], permuted_sub_matrix)
        
        # Store the result in the expected output location
        # Technically it is an output sub-vector
        expected_output[i] = expected_output[i] + result

expected_output = expected_output.reshape(1, 16)
print(f"Expected output after permuted sub-matrices:\n{expected_output}")

# Compare the ideal matrix multiplication with the expected output
print(expected_output == ideal_matmul)

Expected output after permuted sub-matrices:
[[804 924 980 972 772 876 916 892 804 892 916 876 900 972 980 924]]
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]]
